In [116]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
MOUNT_DIR = '/content/drive/MyDrive/BarclaysReviews/'
DATA_DIR = '/content/drive/MyDrive/BarclaysReviews/data/'
MODEL_DIR = '/content/drive/MyDrive/BarclaysReviews/model/'


In [143]:
def saveFileToDrive(path,dataframe):

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    dataframe.to_csv(f)
    print("Saving file : "+path)

In [118]:
final_merged_reviews = pd.read_csv(DATA_DIR+"final_merged_reviews_cleaned.csv")

In [ ]:
pd.set_option('display.max_colwidth', -1)
# final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("reward")][["review_description", "source","rating"]].rating.value_counts()
final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("blue")][["review_description", "source","rating"]].shape

(49, 3)

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
barclays_stop_list = ['app', 'barclays','need','will','pay','work']
stop_words.extend(barclays_stop_list)
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

import re
wnl = WordNetLemmatizer()
def tokenizer(text):
    regex_num_ponctuation = '(\d+)|([^\w\s])'
    regex_little_words = r'(\b\w{1,2}\b)'
    return [wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct ]

def LemmaTokenizer(doc):
      wnl = WordNetLemmatizer()
      regex_num_ponctuation = '(\d+)|([^\w\s])'
      regex_little_words = r'(\b\w{1,2}\b)'
      return [wnl.lemmatize(t) for t in word_tokenize(doc) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t) and not t.is_stop and not t.is_punct]

In [119]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
!pip install HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [123]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [125]:
from sklearn.feature_extraction.text import CountVectorizer 

In [126]:
from bertopic import BERTopic

In [127]:
import time

In [ ]:
def topicModellingBERT(df,columns, textCol = "review_description",min_df=4,max_features = 10000,top_n_words=5,path="",model_name=""):
  print(columns)
  if bool(columns):
    # print("columns")
    for key in columns:
      # print(df.shape)
      df = df[df[key].str.contains(columns[key])]


  min_df = min_df # minimum required occurences of a word, e.g 4
  max_features = max_features # max number of unique words, e.g 10000

  # vectorizer_model = CountVectorizer(
  #               min_df=min_df, 
  #               max_features=max_features, 
  #               tokenizer=tokenizer, # use tokenizer we defined before
  #               ngram_range=(1,2),
  #               stop_words=stop_words) # we use unigram & bigram
  # X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])


  df = df[df[textCol].notnull()]
      
  print('dataet length: ',df.shape,'\n')
  # # we add this to remove stopwords that can pollute topcs
  vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=min_df,max_features=max_features)
  df.reset_index(drop=True, inplace=True)
  model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
  )
  topics, probs = model.fit_transform(df[textCol])
  print(model.get_topic_info())
  
  model.save(path+model_name+'_'+str(int(time.time())))
  return  df,topics, probs, model



In [ ]:
#  topics, probs, model = topicModellingBERT(df=final_merged_reviews,sentiment_col="customer_sentiment",sentiment='NEGATIVE',path=MODEL_DIR,model_name='negative_topic_modelling')
 

In [ ]:

negative_df, negative_topics,  negative_probs,  negative_model = topicModellingBERT(df=final_merged_reviews,columns = {"customer_sentiment":'NEGATIVE'} ,textCol="cleaned_data",path=MODEL_DIR,model_name='negative_topic_modelling')
 

{'customer_sentiment': 'NEGATIVE'}
dataet length:  (6172, 20) 



Batches:   0%|          | 0/193 [00:00<?, ?it/s]

2023-05-01 05:25:26,553 - BERTopic - Transformed documents to Embeddings
2023-05-01 05:25:36,433 - BERTopic - Reduced dimensionality
2023-05-01 05:25:36,926 - BERTopic - Clustered reduced embeddings


   Topic  Count                             Name
0     -1    792            -1_slow_load_bad_open
1      0   4453    0_slow_payment_account_update
2      1    379         1_cheque_branch_bad_slow
3      2    255    2_bad_useless_awful_difficult
4      3    201         3_crash_week_fix_payment
5      4     92  4_slow_slow slow_slow open_open


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
model.visualize_barchart(top_n_topics=12)
#rainbow login info
#word count 


mortage = 49/6.5 k

In [ ]:
model.visualize_barchart(topics=[-1])

In [ ]:
model.visualize_barchart(topics=[-1])

In [ ]:
model.get_topic_info()

,Topic,Count,Name
0,-1,825,-1_update_account_open_new
1,0,1746,0_contactless_payment_payments_bank
2,1,1305,1_account_contactless_card_bank
3,2,415,2_cheque_cheques_photo_paying
4,3,275,3_slow_open_difficult_useless
5,4,240,4_chat_service_call_chat function
6,5,189,5_calendar_view_calendar view_calender
7,6,188,6_contactless_mobile_working_contactless mobile
8,7,173,7_crashes_crashing_keeps_keeps crashing
9,8,151,8_update_slow_updates_every


In [ ]:
model.visualize_hierarchy()

In [ ]:
 import pickle

In [ ]:
with open(MODEL_DIR+'topic_negative_2.pkl', 'wb') as f:
  pickle.dump(topics, f)

In [ ]:
negative_topics = []
with open(MODEL_DIR+"topic_negative_2.pkl", "rb") as input_file:
   negative_topics = pickle.load(input_file)

In [128]:

negative_model = BERTopic.load(MODEL_DIR+"negative_topic_modelling_1682844661") 


In [129]:
negative_df = final_merged_reviews[final_merged_reviews['cleaned_data'].notnull() &final_merged_reviews['customer_sentiment'].str.contains('NEGATIVE') ] 
negative_df.reset_index(drop=True, inplace=True)

In [130]:
negative_model.visualize_barchart(top_n_topics=12)

In [131]:
negative_model.get_topic_info()

,Topic,Count,Name
0,-1,2243,-1_slow_account_bad_payment
1,0,1029,0_contactless_payment_contactless payment_device
2,1,471,1_update_slow_phone_new
3,2,463,2_cheque_photo_picture_camera
4,3,423,3_chat_slow_service_help
5,4,268,4_calendar_calendar_view_notification_crash
6,5,241,5_account_money_slow_year
7,6,229,6_account_register_open_open account
8,7,228,7_crash_login_constantly crash_bad
9,8,134,8_slow_login_account_open


In [134]:
# topics_to_merge = [[5,6],[0,10]]
# topics_to_merge = [[5,6],[0,10]]
topics_to_merge = [[3,6]]
negative_model.merge_topics(negative_df['cleaned_data'], topics_to_merge)
negative_model.get_topic_info()

,Topic,Count,Name
0,-1,2243,-1_slow_account_bad_payment
1,0,1246,0_contactless_payment_contactless payment_slow
2,1,604,1_account_slow_money_login
3,2,496,2_crash_calendar_calendar_view_notification
4,3,471,3_update_slow_phone_new
5,4,463,4_cheque_photo_picture_slow
6,5,423,5_chat_slow_service_help
7,6,123,6_navigate_fscs_open_crash
8,7,103,7_notification_money_payment_spend


In [135]:
# payment = topic_0,topic 10 -l1 mortgage - rewards
# crash = topic 1,4,7 -l1 calender (app or gplay)
# notifications = topic 11,4 - exclude calender
# cheque = topic 2
# login = topic 6, - error > error > customized ,  
# slow = topic 8,5 - 
#customer care -topic 3 chat new chat > crash
# topics_to_merge = [[0,7]]
# negative_model.merge_topics(negative_df['cleaned_data'], topics_to_merge)
# negative_model.get_topic_info()
# negative_model.get_topic_info()
negative_custom_labels = []
negative_custom_labels_temp = negative_model.generate_topic_labels(nr_words=1)
# negative_custom_labels.type
for i in negative_custom_labels_temp:
  negative_custom_labels.append(i.split('_')[1])
negative_model.set_topic_labels(negative_custom_labels)

# negative_custom_labels = negative_model.generate_topic_labels(nr_words=1).apply(lambda x: x.split('_')[1])
# negative_model.set_topic_labels(negative_custom_labels)
negative_model.visualize_barchart(topics=[-1,0,1,2,3,4,5,6],custom_labels=True)
# negative_model.get_topic_freq()

In [147]:
pd.set_option('display.max_colwidth', -1)
# df = negative_df.join(negative_model.get_document_info(negative_df['cleaned_data']))
df[df.Topic==-1].shape



(2243, 27)

In [144]:
saveFileToDrive(DATA_DIR+'Negative_topic_modelling_data.csv',df)

Saving file : /content/drive/MyDrive/BarclaysReviews/data/Negative_topic_modelling_data.csv


In [140]:
negative_model.save(MODEL_DIR+'negative_sentiment_version_2')

In [ ]:
negative_model.visualize_barchart(topics=[-1])

In [ ]:
topics_to_merge = [[-1,2]]
negative_model.merge_topics(negative_df['cleaned_data'], topics_to_merge)
negative_model.visualize_barchart()

In [ ]:
negative_model.visualize_barchart()

In [ ]:
 "tdf,topics, probs, model = topicModellingBERT(df=final_merged_reviews,columns = {"customer_sentiment":'POSITIVE','topic':"1"} ,textCol="cleaned_data",path=MODEL_DIR,model_name='positive_topic_modelling')


{'customer_sentiment': 'POSITIVE'}
dataet length:  (21275, 20) 



Batches:   0%|          | 0/665 [00:00<?, ?it/s]

2023-04-30 13:45:07,104 - BERTopic - Transformed documents to Embeddings
2023-04-30 13:45:25,638 - BERTopic - Reduced dimensionality
2023-04-30 13:45:34,154 - BERTopic - Clustered reduced embeddings


    Topic  Count                                               Name
0      -1   9269          -1_excellent_service_convenient_brilliant
1       0   3452                   0_account_money_payment_transfer
2       1    564   1_excellent excellent_excellent_positive_overall
3       2    452                2_great great_great_yeah_like great
4       3    416        3_quick quick_quick_quick issue_issue quick
5       4    363                         4_cheque_photo_picture_bad
6       5    347               5_great great_great_trouble_actually
7       6    304          6_intuitive_seamless_fast service_service
8       7    290         7_perfect_superb_amazing_brilliant perfect
9       8    282          8_experience_service service_service_love
10      9    278                   9_super_love_fine_efficient safe
11     10    265                         10_tin_save slow_save_love
12     11    257         11_service_customer_staff_customer service
13     12    241            12_device_transfer_p

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
model.visualize_topics()

In [ ]:
model.reduce_topics(df['cleaned_data'],nr_topics=8)

2023-04-30 13:29:48,851 - BERTopic - Reduced number of topics from 42 to 8


In [ ]:
model.visualize_barchart()